# Creating our Library Database

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect(':memory:')

In [3]:
cursor = conn.cursor()

In [5]:
cursor.execute('''
CREATE TABLE Authors (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL
)
''')

In [6]:
cursor.execute('''
CREATE TABLE Books (
  id INTEGER PRIMARY KEY,
  title TEXT NOT NULL,
  author_id INTEGER,
  year INTEGER,
  rating REAL,
  FOREIGN KEY (author_id) REFERENCES Authors (id)
)
''')

In [7]:
cursor.execute('''
CREATE TABLE Borrowers (
  id INTEGER PRIMARY KEY,
  book_id INTEGER,
  borrowers_name TEXT NOT NULL,
  borrow_date TEXT NOT NULL,
  FOREIGN KEY (book_id) REFERENCES Books (id)
)
''')

In [8]:
authors_data = [
    (1,'J.K Rowling'),
    (2, "Geroge R.R Martin"),
    (3, "J.R.R Tolkien")
]

In [9]:
cursor.executemany('''
INSERT INTO Authors (id,name) VALUES (?,?)
''', authors_data
)

In [10]:
books_data = [
    (1,'Harry potter and the Philosopher stone',1,1997,4.8),
    (2,'Harry potter and the Chamber of Secrets',3,1998,4.7),
    (3,'Harry potter and the Prisoner of Azkaban',1,1999,4.6),
    (4,'Game of thrones',2,1996,4.9),
    (5,'The Silmarillion',3,1977,4.8)
]

In [11]:
cursor.executemany('''
INSERT INTO Books (id,title,author_id,year,rating) VALUES (?,?,?,?,?)
''', books_data
)

In [12]:
borrowers_data = [
    (1,1,'bob','2023-08-01'),
    (2,2,'alice','2023-08-02'),
    (3,3,'charlie','2023-08-03'),
    (4,4,'david','2023-08-04'),
    (5,5,'eve','2023-08-05'),
    (6,6,'frank','2023-08-06'),
]

In [13]:
cursor.executemany('''
INSERT INTO Borrowers (id,book_id,borrowers_name,borrow_date) VALUES (?,?,?,?)
''', borrowers_data
)

In [14]:
conn.commit()

#Basic SQL Queries

## Can you give me a complete list of all the books we have in our database?

In [15]:
books_df = pd.read_sql_query("SELECT * FROM Books", conn)
books_df

,id,title,author_id,year,rating
0,1,Harry potter and the Philosopher stone,1,1997,4.8
1,2,Harry potter and the Chamber of Secrets,3,1998,4.7
2,3,Harry potter and the Prisoner of Azkaban,1,1999,4.6
3,4,Game of thrones,2,1996,4.9
4,5,The Silmarillion,3,1977,4.8


## I’m particularly interested in the books written by J.K. Rowling. Can you filter out just those books for me?

In [16]:
jk_books = pd.read_sql_query("SELECT * FROM Books WHERE author_id = 1", conn)
jk_books

,id,title,author_id,year,rating
0,1,Harry potter and the Philosopher stone,1,1997,4.8
1,3,Harry potter and the Prisoner of Azkaban,1,1999,4.6


## Can you provide a list that shows each book along with the name of the author who wrote it?

In [17]:
books_with_authors = pd.read_sql_query("SELECT Books.title, Authors.name FROM Books JOIN Authors ON Books.author_id = Authors.id", conn)
books_with_authors

,title,name
0,Harry potter and the Philosopher stone,J.K Rowling
1,Harry potter and the Chamber of Secrets,J.R.R Tolkien
2,Harry potter and the Prisoner of Azkaban,J.K Rowling
3,Game of thrones,Geroge R.R Martin
4,The Silmarillion,J.R.R Tolkien


## I need to know how many books each author has written. Can you get me that information?

In [36]:
books_count = pd.read_sql_query('''
SELECT Authors.name as author_name,


count(*) as books_count FROM Books



JOIN Authors ON Books.author_id = Authors.id


GROUP BY Authors.name

ORDER BY books_count DESC
LIMIT 1
''', conn
)

In [19]:
books_count

,author_name,books_count
0,J.R.R Tolkien,2


# Advanced SQL Queries

## I’ve heard that J.R.R. Tolkien is one of our most popular authors. Could you pull up all of his books from the database?

In [26]:
most_popular = pd.read_sql_query('''
SELECT * FROM Books WHERE author_id = (
  SELECT id FROM Authors WHERE name = 'J.R.R Tolkien'
)
''', conn
)

In [27]:
most_popular

,id,title,author_id,year,rating
0,2,Harry potter and the Chamber of Secrets,3,1998,4.7
1,5,The Silmarillion,3,1977,4.8


## Can you show me the ranking of books by each author based on the year they were published? I’d like to see the order in which they were written.

In [32]:
ranked_books = pd.read_sql_query('''
SELECT title, author_id,
RANK() OVER (PARTITION BY author_id ORDER BY year) AS ranking
FROM Books
''', conn
)

In [33]:
ranked_books

,title,author_id,ranking
0,Harry potter and the Philosopher stone,1,1
1,Harry potter and the Prisoner of Azkaban,1,2
2,Game of thrones,2,1
3,The Silmarillion,3,1
4,Harry potter and the Chamber of Secrets,3,2


## I’m interested in our top-performing books. Can you pull up the top 3 highest-rated books in our collection?

In [34]:
top_books = pd.read_sql_query('''
WITH TopBooks AS(
SELECT title, rating FROM Books ORDER BY rating DESC LIMIT 3
)
SELECT * FROM TopBooks
''', conn
)

In [35]:
top_books

,title,rating
0,Game of thrones,4.9
1,Harry potter and the Philosopher stone,4.8
2,The Silmarillion,4.8


# SQL ASSIGNMENT

## Get the top 3 most borrowed books using a subquery

In [38]:
query = """
SELECT title, author_id, borrow_count
FROM (
    SELECT b.title, b.author_id, COUNT(br.book_id) AS borrow_count
    FROM Books b
    JOIN Borrowers br ON b.id = br.book_id
    GROUP BY b.id
) subquery
ORDER BY borrow_count DESC
LIMIT 3;
"""
cursor.execute(query)
top_borrowed_books = cursor.fetchall()
df_top_borrowed_books = pd.DataFrame(top_borrowed_books, columns=["Title", "Author ID", "Borrow Count"])
print(df_top_borrowed_books)


                                      Title  Author ID  Borrow Count
0    Harry potter and the Philosopher stone          1             1
1   Harry potter and the Chamber of Secrets          3             1
2  Harry potter and the Prisoner of Azkaban          1             1


## Rank books by an author using a window function.

In [39]:
query = """
SELECT title, author_id, rating,
       RANK() OVER (PARTITION BY author_id ORDER BY rating DESC) AS rank
FROM Books;
"""
cursor.execute(query)
ranked_books = cursor.fetchall()
df_ranked_books = pd.DataFrame(ranked_books, columns=["Title", "Author ID", "Rating", "Rank"])
print(df_ranked_books)


                                      Title  Author ID  Rating  Rank
0    Harry potter and the Philosopher stone          1     4.8     1
1  Harry potter and the Prisoner of Azkaban          1     4.6     2
2                           Game of thrones          2     4.9     1
3                          The Silmarillion          3     4.8     1
4   Harry potter and the Chamber of Secrets          3     4.7     2


## List all customers who have made a purchase in the last 6 months.

In [41]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
six_months_ago = (datetime.now() - timedelta(days=6 * 30)).strftime('%Y-%m-%d')
query = """
SELECT DISTINCT borrowers_name
FROM Borrowers
WHERE borrow_date >= ?
"""
cursor.execute(query, (six_months_ago,))
recent_borrowers = cursor.fetchall()
df_recent_borrowers = pd.DataFrame(recent_borrowers, columns=["Borrower Name"])
print(df_recent_borrowers)


Empty DataFrame
Columns: [Borrower Name]
Index: []


## Identify the book that has been borrowed by the most unique members.

In [42]:
query = """
SELECT b.title, COUNT(DISTINCT br.borrowers_name) AS unique_borrowers
FROM Books b
JOIN Borrowers br ON b.id = br.book_id
GROUP BY b.id
ORDER BY unique_borrowers DESC
LIMIT 1;
"""
cursor.execute(query)
most_borrowed_book = cursor.fetchall()
df_most_borrowed_book = pd.DataFrame(most_borrowed_book, columns=["Book Title", "Unique Borrowers"])
print(df_most_borrowed_book)


         Book Title  Unique Borrowers
0  The Silmarillion                 1
